<a href="https://colab.research.google.com/github/Nedu21/Pytorch-deep-learning-projects-/blob/main/Logistic_Regression_on_the_Iris_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression on the Iris Dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

Load the data

In [2]:
iris = load_iris()
features = iris.data
labels = iris.target
print(features.shape)
print(labels.shape)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(150, 4)
(150,)


In [3]:
# Convert the x and y values into tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [4]:
# TensorDataset provides a simple way to wrap one or more tensors into a dataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [5]:
# Load the dataset using a DataLoader
batch_size = 12

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# Build the model
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, output_size)
    )

  def forward(self, x):
    logits = self.linear_relu_stack(x)
    return logits

In [7]:
input_size = X_train.shape[1]
hidden_size = 32
output_size = 3

model = NeuralNet(input_size, hidden_size, output_size)
print(model)

NeuralNet(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [9]:
print(len(train_dataloader.dataset))
print(len(train_dataloader))

120
10


In [10]:
def train(dataloader, model, criterion, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    # Compute prediction error
    yhat = model(X)
    loss = criterion(yhat, y)
    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 1 == 0:
      loss, current = loss.item(), (batch + 1)*len(X) # ie for batch 10 current = (10 + 1) * 10 (10 is the batch size)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [11]:
def test(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            yhat = model(X)
            loss = criterion(yhat, y)
            test_loss += loss.item()
            # Compares predictions (argmax) to true labels (y), counts matches (booleans to floats 1/0), then sums to get total correct in the batch.
            correct += (yhat.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}% Avg Loss: {test_loss:>7f} \n')

In [12]:
epochs = 10

for epoch in range(epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  train(train_dataloader, model, criterion, optimizer)
  test(test_dataloader, model, criterion)

print('Done!')

Epoch 1
-------------------------------
loss: 1.350471  [   12/  120]
loss: 1.253924  [   24/  120]
loss: 1.266242  [   36/  120]
loss: 1.162511  [   48/  120]
loss: 1.104085  [   60/  120]
loss: 1.194379  [   72/  120]
loss: 1.111503  [   84/  120]
loss: 1.046549  [   96/  120]
loss: 0.988817  [  108/  120]
loss: 1.047746  [  120/  120]
Test Error: 
 Accuracy: 56.7% Avg Loss: 0.970712 

Epoch 2
-------------------------------
loss: 1.092871  [   12/  120]
loss: 1.137251  [   24/  120]
loss: 0.911013  [   36/  120]
loss: 1.083982  [   48/  120]
loss: 1.224849  [   60/  120]
loss: 0.971531  [   72/  120]
loss: 1.018491  [   84/  120]
loss: 1.138237  [   96/  120]
loss: 1.087401  [  108/  120]
loss: 0.922851  [  120/  120]
Test Error: 
 Accuracy: 56.7% Avg Loss: 0.891911 

Epoch 3
-------------------------------
loss: 1.032872  [   12/  120]
loss: 0.891591  [   24/  120]
loss: 1.111849  [   36/  120]
loss: 0.928922  [   48/  120]
loss: 1.031097  [   60/  120]
loss: 0.918311  [   72/  120